#### Neural network classification
- binary classification (example: dog or cat)
- multiclass classification (more than 2 classes)

#### What we're going to cover

- Architecture of a neural network classification model
- Input shapes and output shapes of a classification model (features and labels)
- Creating custom data to view, fit on and predict on
- Steps in modelling:
    - creating a model
    - setting up loss function and optimizer
    - creating a training loop
    - evaluating a model
- Saving and loading a models
- Harnessing the power of non-linearity
- Different classification evaluation methods    

In [ ]:
# batch size (32 is commonly used in practice)